In [11]:
%load_ext autoreload
%autoreload 2

from tqdm import tqdm
from math import sqrt,erf
from multiprocessing import Pool
from itertools import product, repeat
from matplotlib import pyplot as plt
from matplotlib import animation
import matplotlib as mpl
import numpy as np
from varname import nameof
from itertools import product
from matplotlib.cm import get_cmap
import colorcet as cc
import subprocess
import os, shutil
from time import time
import functools
from scipy import special
import scipy
from IPython.display import display, clear_output
import fermionise as fermionise
import eigenstateRG as eigRG


plt.style.use('ggplot')
markers = ["o", "X", "P", "p", "*"]
cols = [p['color'] for p in plt.rcParams['axes.prop_cycle']]
plt.rc('text.latex', preamble=r'\usepackage{amsmath}\usepackage{braket}\usepackage{nicefrac}')
plt.rcParams.update({'font.size': 30,
                     'figure.figsize': (11,7),
                     'axes.facecolor': 'white',
                     'axes.edgecolor': 'lightgray',
                     "figure.autolayout": 'True',
                     'axes.xmargin': 0.03,
                     'axes.ymargin': 0.05,
                     'axes.grid': False,
                     'axes.linewidth': 5,
                     'lines.markersize': 15,
                     'text.usetex': True,
                     'lines.linewidth': 8,
                     "legend.frameon": True,
                     "legend.framealpha": 0.7,
                     "legend.handletextpad": 1,
                     "legend.edgecolor": "black",
                     "legend.handlelength": 1,
                     "legend.labelspacing": 0,
                     "legend.columnspacing": 1,
                     "legend.fontsize": 35,
                    })
linestyles = ["-", "--", ":"]
bbox = dict(boxstyle="round", facecolor="lightgray")

D0 = 1
deltaD = 0.001
plt.plot([], [])
plt.show()
clear_output()

In [12]:
def get_RG_flow(J0_by_D0, Ub_by_J=0, plot=False):
    """ Returns the flow of couplings in the form of two ndarrays J and D.
    Each ndarray is in ascending order of the bandwidth. """
    J0 = D0 * J0_by_D0
    Ub = - Ub_by_J * J0
    omega = -D0/2
    
    ### initialise arrays with UV values
    D = [D0]
    J = [J0]
    
    ### apply URG transformations until bandwith vanishes
    ### or J reduces to zero.
    while D[-1] >= deltaD and J[-1] >= 0:
        
        ### URG equation
        deltaJ = - J[-1] * (J[-1] + 4 * Ub) / (omega - D[-1]/2 + J[-1]/4) * deltaD
        
        ### Check if denominator has changed sign, 
        ### then append renormalised values to array
        if (omega - (D[-1] - deltaD)/2 + (J[-1] + deltaJ)/4) * (omega - D0/2 + J0/4) > 0:
            D.append(D[-1] - deltaD)
            J.append(J[-1] + deltaJ)
        else:
            break
    
    ### plot values
    if plot:
        plt.plot(np.array(D)/D0, np.array(J)/J0, marker="o")
        plt.xlabel(r"$D/D_0$")
        plt.ylabel(r"$J/J_0$")
    return np.flip(D), np.flip(J)

In [16]:
def KondoMERG(J0_by_D0, num_entangled, both_sectors=False):
    D, J = get_RG_flow(J0_by_D0)
    omega = -D0/2
    time_list = []
    # for num_IOMs in range(2, 15, 2):
    for num_IOMs in [2]:
        t = time()
        Ek = np.linspace(D[0] - deltaD * num_entangled, D[0], num_entangled)
        if both_sectors:
            alpha_arr = np.repeat([Ji / (2 * (omega - Di/2 + Ji/4)) for Ji, Di in zip(J, D)], 2)[:num_IOMs]
            IOMconfigs = [1 if i % 2 == 0 else 0 for i in range(num_IOMs)]
            print (IOMconfigs)
        else:
            alpha_arr = [Ji / (2 * (omega - Di/2 + Ji/4)) for Ji, Di in zip(J, D)][:num_IOMs]
            IOMconfigs = [0] * num_IOMs
        # print (Ek)
        init_couplings = [Ek, J[0], 0]
        decomposition_arr = eigRG.getWavefunctionRG(init_couplings, alpha_arr, num_entangled, num_IOMs, 
                                                   IOMconfigs, fermionise.getKondoHamiltonian, eigRG.getEtaKondo, silent=False
                                                  )
        print (decomposition_arr)
        computation_results = eigRG.computations(decomposition_arr,
                                                {"VNE": [2, 3],
                                                }
                                               )

KondoMERG(0.5, 2, True)

[1, 0]
G-state energy: [-2.78671074]
↓|0|↑		↓|↑|0		↑|0|↓		↑|↓|0
0.5		0.5		-0.5		-0.5



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 7731.44it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14/14 [00:00<00:00, 1052.98it/s]


[{'010010': 0.499790766, '011000': 0.5002091465, '100001': -0.499790766, '100100': -0.5002091465}, {'01001011': 0.0005112831884372641, '01100011': 0.0005117111893740002, '10000111': -0.0005112831884372641, '10010011': -0.0005117111893740002, '01101001': 0.0001479197701661339, '01011010': -0.3535531823857567, '01001110': 0.3534052626155906, '10011001': 0.3535531823857567, '10001101': -0.3534052626155906, '01110010': 0.3537011021559229, '01100110': 0.3535531823857567, '10110001': -0.3537011021559229, '10100101': -0.3535531823857567, '10010110': 0.0001479197701661339}, {'0100101100': 4.677534352867797e-07, '0110001100': 4.6814499697508037e-07, '1000011100': -4.677534352867797e-07, '1001001100': -4.6814499697508037e-07, '0110100100': 1.3532614059445032e-07, '0101101000': -0.00032345228506922274, '0100111000': 0.00032331695892862827, '1001100100': 0.00032345228506922274, '1000110100': -0.00032331695892862827, '0111001000': 0.0003235876112098172, '0110011000': 0.00032345228506922274, '101100


Computing VNE: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 1085.20it/s]


In [ ]:
states = [[1,0], [1,1], [0,1]]
coeffs = [0.1, 0.2, 0.3]


In [ ]:
def func(args):
    x, n = args
    return x**(3**3**2)

num = 1000
power = 10
t = time()
results1 = [func((x, power)) for x in range(1, num)]
print (time() - t)
t = time()
results2 = Pool().map(func, [(x, power) for x in range(1, num)], chunksize=50)
# print (workers[0].get())
# print (workers[1])
# results = [worker.get() for worker in workers]
print (time() - t)

In [ ]:
def func(x):
    return np.arange(3)**x**4

p = Pool()
result = p.apply_async(func, (2,)).get()
print(result)

In [ ]:
import os
os.cpu_count()

In [ ]:
d1 = dict([("a", 1), ("b", 2)])
d2 = dict([("a", 0), ("b", 0), ("c", 0)])
d2.update(d1)
d2